### Import Stuff

In [2]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial import distance
import scipy.constants as constants
import random
from collections import Counter

### Particle Simulation Class

In [3]:
class particle:
    # a class to simulate the movement of radon/polonium ions in a hemispherical vessel with an applied electric field.
    # the flag (set in the simulation function) shows if a the corresponding particle did decaye on its way to the diode or if it left the vessel
    #     flag = 0  => particle did decay
    #     flag = 1  => particle did not decay
    #     flag = -1 => particle left vessel volume
    
    
    
    
    
    def __init__(self, dt, textfile, mobility, loops):
        self.time = 0. # total drifttime
        self.dt = dt # timesteps
        self.efield = self.txt_to_sa(textfile)
        self.field_strength = [0,0]
        self.mobility = mobility
        self.loops = loops
        self.result_tupel = []
        self.position = [1,1]
        self.ef = [list(x) for x in self.efield[['r','z']]]
        self.pathlist =[]
        self.diff_velocity = np.sqrt(8 * constants.k * 300 / (np.pi * 222 * constants.physical_constants['atomic mass constant'][0]))
        self.pathlist_fail = []
        self.init_x = []
        self.init_y = []
        self.init_z = []
        self.timelist = []
    
    def txt_to_sa(self,file):
        #create a structured array from a txt file
        with open(file) as f:
            content = f.readlines()
        content = [line.split() for line in content
                  if not '%' in line]
        content = [tuple(np.float128(y) for y in x) for x in content]

        struc_array = np.array(content, np.dtype([('r',np.float128),('z',np.float128),('er',np.float128),('ez',np.float128)]))
        return struc_array

    
    def set_init_position(self):
        # create inital position
        x_pos = 0.0781*random.random()
        y_pos = 0.0781*random.random()
        z_pos = 0.0891*random.random()
        while ((z_pos < (0.0781)) and  ((z_pos-0.0781)**2 + x_pos**2 + y_pos**2 > 0.0781**2)) or (x_pos**2 + y_pos**2 > 0.0781**2):
            random_angle = 2*np.pi*random.random()
            x_pos = 0.0781*random.random()*np.cos(random_angle)
            y_pos = x_pos*np.tan(random_angle)
            z_pos = 0.0891*random.random()
        self.x = x_pos
        self.y = y_pos
        self.z = z_pos
        self.init_x.append(self.x)
        self.init_y.append(self.y)
        self.init_z.append(self.z)
        self.r = np.sqrt(x_pos**2 + y_pos**2)
        self.init_position = np.array([x_pos, y_pos, z_pos])
        self.position = np.array([self.r, z_pos])
    
    def update_nearest_point(self):
        # get the nearest point in the e field txt

        closest_index = distance.cdist([self.position], self.ef).argmin()
        self.nearest_point = self.ef[closest_index]
    
    def update_efield(self):
        # search the e field for position
        self.field_strength = -1*np.array([self.efield[(self.efield['r']==self.nearest_point[0]) & (self.efield['z']==self.nearest_point[1])][0][2],
                              self.efield[(self.efield['r']==self.nearest_point[0]) & (self.efield['z']==self.nearest_point[1])][0][3]])
    
    
    def set_diffusion(self):
        random_theta = 2 * np.pi * random.random()
        random_phi = 2 * np.pi * random.random()
        self.diff_vel_vec = self.diff_velocity * np.array([np.sin(random_theta)*np.cos(random_phi), np.cos(random_theta)])
        
    
    def update_drift(self):
        self.drift = self.mobility * self.field_strength * self.dt
        
    def update_position(self):
        self.position = self.position + self.drift + self.diff_vel_vec * self.dt
        self.r = self.position[0]
        self.z = self.position[1]
    
    def update_time(self):
        self.time = self.time + self.dt
    
    def append_result(self):
        self.result.append([self.init_position[0], self.init_position[1], self.init_position[2], self.time, self.flag])
    
    def return_result(self):
        # most useless part of this class
        return self.result
    
    def simulate(self):
        # run the simulation
        
        
        for i in tqdm(range(self.loops)):
            self.time = 0.
            self.path = []
            fail = False
            fail_1 = False
            self.set_init_position()
            self.path.append(self.position)
            dead = False
            j = 0
            while (self.position[1] <= 0.0889) or (self.position[0] > 0.0045):
                self.update_nearest_point()
                self.update_efield()
                self.set_diffusion()
                self.update_drift()
                self.update_position()
                self.path.append(self.position)
                self.update_time()
                if (((self.position[1] > 0.0889) and (self.position[0] > 0.05)) or
                   (self.position[0] > 0.0781) or
                   ((self.position[1] < 0.0781) and
                   ((self.position[1]-0.0781)**2 + self.position[0]**2 > 0.0781**2))):
                    self.flag = -1
                    dead = True
                    break
            self.create_decay_time()
            if (self.time >= self.decay_time) and (not dead):
                self.flag = 0
            elif (not dead):
                self.flag = 1
            self.timelist.append(self.time)
            self.result_tupel.append((self.x,self.y,self.z,self.time,self.flag))
            #self.append_result()
            self.pathlist.append(self.path)
        self.result_to_structured_array()
        self.calc_efficiency()
        self.return_result()
    
    def create_init(self):
        self.path_x = []
        self.path_y = []
        for i in range(loops):
            self.set_init_position()
            self.path_x.append(self.position[0])
            self.path_y.append(self.position[1])
    
    def create_decay_time(self):
        # create the duration for the particle to exist
        
        self.decay_constant = 3.1*60/np.log(2)
        random_number = random.random()
        self.decay_time = 10*60*random.random()
        p_exp = np.exp(-self.decay_time/self.decay_constant)
        while random_number >= p_exp:
            random_number = random.random()
            self.decay_time = 10*60*random.random()
            p_exp = np.exp(-self.decay_time/self.decay_constant)
            
    def result_to_structured_array(self):
        #wip
        data_dtype = np.dtype([('init x',np.float64),('init y',np.float64),('init z',np.float64),('time', np.float64),('flag',np.int8)])
        self.result = np.array(self.result_tupel,data_dtype)
        
    def calc_efficiency(self):
        n_counts = Counter(self.result['flag'])
        if 0 in n_counts:
            n_decayed = n_counts[0]
        else:
            n_decayed = 0
        
        if -1 in n_counts:
            n_lost = n_counts[-1]
        else:
            n_lost = 0
            
        if 1 in n_counts:
            n_alive = n_counts[1]
        else:
            n_alive = 0
        self.efficiency = n_alive / (n_alive + n_lost + n_decayed)
        
    def do_reset(self):
        # reset all list etc
        
        self.result_tupel = []
        self.pathlist =[]
        self.pathlist_fail = []
        self.init_x = []
        self.init_y = []
        self.init_z = []
        self.timelist = []

### Simulation

In [4]:
sim = particle(dt = 10**(-6), textfile='hemisphere.txt', mobility = 0.16, loops = 100)

In [8]:
sim.simulate()

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


In [9]:
print(sim.efficiency)

1.0


In [7]:
sim.loops = 1